- Kaggle에 업로드 된 movies 데이터 셋 사용 -> dtm_tfidf를 구축한다.
- 여러 distance function의 성능 비교.
- np.argsort()


In [ ]:
# install and import the libraries needed
# this script is heavily inspired by Wikidocs
!pip3 install scikit-learn
!pip3 install gdown
from typing import Tuple, List
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import os
import gdown # for downloading the data stored in google drive

In [ ]:
# 변수 저장
MOVIES_META_CSV_URL = "https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download"
MOVIES_META_CSV = "./movies_metadata.csv"
DOC_SIZE = 2000
TOP_N = 30

In [ ]:
# 데이터 다운받기
gdown.download(url=MOVIES_META_CSV_URL, output=MOVIES_META_CSV, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download
To: /content/movies_metadata.csv
34.4MB [00:00, 56.3MB/s]


'./movies_metadata.csv'

In [ ]:
# 데이터 보기
movies_df = pd.read_csv(MOVIES_META_CSV, low_memory=False)
movies_df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# 데이터프레임 사이즈 조절하기
movies_df = movies_df.head(DOC_SIZE)

# 결측치 채우기
movies_df['overview'] = movies_df['overview'].fillna('')

In [ ]:
# 1
# CountVectorizer와 TfidVectorizer을 이용하여 dtm과 dtm_tfidf를 구축한다
count = CountVectorizer(binary=False)
tfidf = TfidfVectorizer()
dtm: np.ndarray = count.fit_transform(movies_df['overview']).toarray()
dtm_tfidf: np.ndarray = tfidf.fit_transform(movies_df['overview']).toarray()

print(dtm)
print(dtm.shape)
print(dtm_tfidf)
print(dtm_tfidf.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2000, 14195)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2000, 14195)


In [ ]:
# 코사인 유사도 -> 벡터화 시키는 것(?) 벡터화 돼있음(?)
sims_cosine = cosine_similarity(dtm)
sims_cosine_tfidf = cosine_similarity(dtm_tfidf)

print(sims_cosine.shape)
print(sims_cosine_tfidf.shape)

(2000, 2000)
(2000, 2000)


In [ ]:
# 맨하탄 거리 (시간이 조금 걸림. 효율적이지 않은 것일까?)
dists_manhattan_tfidf = manhattan_distances(dtm_tfidf)

print(dists_manhattan_tfidf.shape)

(2000, 2000)


In [ ]:
# 유클리디안 거리
dists_euclidean_tfidf = euclidean_distances(dtm_tfidf)

print(dists_euclidean_tfidf.shape)

(2000, 2000)


In [19]:
# 제목들을 검색하기 위해 중복행을 제거한다
indices = pd.Series(data=movies_df.index, index=movies_df['title']).drop_duplicates()
indices

title
Toy Story                         0
Jumanji                           1
Grumpier Old Men                  2
Waiting to Exhale                 3
Father of the Bride Part II       4
                               ... 
Tex                            1995
Tron                           1996
Swing Kids                     1997
Halloween: H20                 1998
L.A. Story                     1999
Length: 2000, dtype: int64

In [11]:
batman_idx = indices["Batman"]
batman_return_idx = indices["Batman Returns"]
toy_story_idx = indices["Toy Story"]

print(sims_cosine_tfidf[batman_idx, batman_return_idx])
print(sims_cosine_tfidf[batman_idx, toy_story_idx])

0.19155692273625355
0.01586093997388227


In [12]:
print(type(indices))

<class 'pandas.core.series.Series'>


In [20]:
def get_recommendations(title: str,
                        # dists X
                        sims: np.ndarray,
                        indices: pd.Series,
                        data: pd.DataFrame) -> List[Tuple[str, float]]:

  global TOP_N

  #2
  # 루프 사용하지 않기
  # 점수와 제목을 추천하기 위해 np.argsort()[::-1] 사용하기
  title_idx = indices[title] # 제목의 인덱스들
  sims_to_title = sims[title_idx, :] # 유사도 (2000,)
  sims_to_title_sorted: List[str] = np.sort(sims_to_title)[::-1] # 제목별 내림차순 (list형)
  indices_sorted = np.argsort(sims_to_title)[::-1] # 제목별 내림차순의 인덱스 (ndarray형)
  titles_sorted: List[float] = data['title'][indices_sorted] # 인덱스에 해당하는 제목을 리스트로 변환

  results: List[Tuple[str, float]] = [
          (title, score)
          for title, score in zip(titles_sorted, sims_to_title_sorted)
  ]

  return results[:TOP_N]

In [21]:
# results와 tfidf를 비교해보자
print("< DTM + cosine sim >")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine, indices, movies_df)):
  print(idx, title, score)

print("\n")

print("< TFIDF + cosine sim >")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)

< DTM + cosine sim >
0 Batman 1.0000000000000002
1 Mary Shelley's Frankenstein 0.45268314226448536
2 The Leopard Son 0.4468418247369901
3 The Last Emperor 0.44633201988370275
4 The Confessional 0.4436069753671345
5 Sweet Nothing 0.43970404417590525
6 Conan the Barbarian 0.4375949744936837
7 The Children Are Watching Us 0.43193421279068006
8 The Search for One-eye Jimmy 0.43092875104518874
9 Stalingrad 0.42857142857142855
10 North Star 0.42839304295347314
11 I Can't Sleep 0.42766686606638943
12 The Rocketeer 0.42721034696687243
13 Days of Thunder 0.4269760205378603
14 The Remains of the Day 0.42525863589985735
15 My Life and Times With Antonin Artaud 0.4198925212700719
16 The Replacement Killers 0.4190262407031392
17 Ed Wood 0.4164633650362828
18 Commandments 0.41475753100312657
19 Return of the Jedi 0.4139469470433775
20 And the Band Played On 0.41226488477260886
21 Mad Dog Time 0.4114755998989118
22 Child's Play 2 0.41099746826339323
23 The Education of Little Tree 0.4103049699311091


In [22]:
# 3
# 다른 유사도 방법들과도 비교해보자
# dists와 sims는 다름
print("< TFIDF + cosine sim >")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)

print("\n")

print("< TFIDF + euclidean dists >")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_euclidean_tfidf, indices, movies_df)):
  print(idx, title, score)

print("\n")

print("< TFIDF + manhattan dists >")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_manhattan_tfidf, indices, movies_df)):
  print(idx, title, score) 

< TFIDF + cosine sim >
0 Batman 1.0000000000000002
1 Batman Returns 0.19155692273625355
2 Batman & Robin 0.1758295995796183
3 Batman Forever 0.14581071757552017
4 Cry, the Beloved Country 0.11109312245823069
5 B. Monkey 0.10544235266596627
6 The Shadow 0.10514679108660141
7 Sweet Nothing 0.10035634826245518
8 A Month by the Lake 0.10023026852720505
9 Chairman of the Board 0.09920814012584121
10 The War at Home 0.0988387365764903
11 Dune 0.09875401691372578
12 Mediterraneo 0.09803176049698262
13 Return of the Jedi 0.09217825522595026
14 The Pompatus of Love 0.09085969169749389
15 Cyclo 0.08667634812405352
16 Commandments 0.08651635230910235
17 Apocalypse Now 0.08609774357728836
18 Friday the 13th: A New Beginning 0.08605652456139677
19 Dangerous Game 0.08604236366845823
20 Highlander 0.08602706777821761
21 Metropolis 0.08398384733774258
22 Somebody Is Waiting 0.08258337613352316
23 M 0.08193657697542574
24 Mrs. Miniver 0.0811774410090339
25 The Apostle 0.08031383520515435
26 Love & Huma

# cosine >> euclidean, manhatten인 이유는?

1. 코사인 유사도의 경우, 벡터의 크기를 고려하지 않고 각도만을 고려한다. 차원의 저주의 영향을 덜 받는다

2. 분석하고자하는 데이터가 다차원 벡터일 경우, 벡터 간 코사인 각도를 사용할 시 더욱 정확한 유사도를 측정할 수 있다

3. 줄거리의 길이가 2 - 178. 코사인 유사도는 단어의 구성비율의 유사도만을 보기 때문에 줄거리의 길이가 다른 현재 문제에 적합하다. 벡터의 크기로 비교하게 되면 곤란. 길이가 달라도 구성비율이 비슷하다면 줄거리가 비슷한 것이다. 그런 상황을 유클리드나 맨해튼은 고려하지 못한다. 벡터의 크기를 고려하기 때문이다. 그래서 현 문제에는 코사인 거리함수의 성능이 좋다.

# 일반적으로 고차원 벡터인 경우, manhattan이 euclidean보다 성능이 좋은 것으로 알려져 있습니다 (슬라이드). 하지만 이 문제에서는 euclidean > manhattan 입니다. 그 이유는?

- 논문에서 일반적이라 함은 정규분포임
- 우리가 만든 tfidf 벡터의 성분은 keword는 높게, 중요하지 않은 것은 낮게
(이상치를 만듦. 이상치 증폭: 편차의 제곱)
- 희소성 때문에(희소성은 차원의 저주의 결과). 
- 고차원으로 갈 수록 거리의 개념이 옅어진다. 모든 벡터가 커지므로.
- 유클리디안은 outlier값을 더 증폭시킨다
- 맨해튼은 제곱을 하지 않기 때문에 outlier값에 강건하며 차원의 저주를 줄일 수 있다.